# 0. 이번 시간에는 'Data Trust(데이터 신뢰성)'에 대해서 실습을 진행해보려고 한다.

Data Trust, 즉 데이터를 신뢰한다는 의미는 무엇을 의미하는 걸까?

※**Data Integrity(데이터 무결성)**:데이터가 생성,저장,전송,처리되는 모든 과정에서 '정확성','완전성','일관성'을 유지하며, 손상,변경,삭제,유출이 되지않도록 보장하는 것

->즉, 간단하게 말하면, **데이터의 일관성이 유지**되는 것을 의미함.

->여기서, 데이터의 일관성은, **내가 최근에 처리한 데이터 그 상태 그대로 유지함**을 말함.



---



※**(Wrong!!) 데이터 무결성(Integrity) vs 데이터 일관성(Consistency)**

내가 위에서 말한 '데이터의 일관성이 유지되는 것'은 말 그대로 '일관성(Consistency)'와 관련이 있지, 엄밀히 말하면 무결성은 아니다.

한 마디로 요약하자면, **'데이터 무결성(Integrity)' = '데이터가 변질되지 않음(최근에 처리한 데이터 그 상태 그대로 유지함)'** 이고 **'데이터 일관성(Consistency)' = '데이터가 서로 일치함(데이터들끼리의 비교)'**이다.

※그럼에도 불구하고, 데이터 무결성(Integrity) 및 데이터 일관성(Consistency)은 매우 뜻이 유사해보인다.

->밑에 '계층별 Audit 전략'의 각 단계 예시들을 확인해보면 조금 구분이 쉬움을 알 수 있듯이, '**택배 상자의 배송 과정**'을 비유로 들어서 이해하면 편하다.



```
데이터 무결성(데이터가 변치 않음): 택배 배송 과정에서 택배가 변하지 않음. -> 하지만, 여기에 '최소한의 기준("박스는 네모나야 한다(형식)", "주소는 필수다(Not Null)" 같은 기본 규칙 -> 제약 조건(constraint))'은 있어야 한다.
데이터 일관성(데이터가 서로 일치함): 배송 후에 비교를 해봤더니 데이터가 일치하느냐? -> 정말 말 그대로, '세부적인 두 데이터의 값'을 비교하는 것임.

->즉, 데이터 무결성(integrity)는 상대적으로 전체적인 것을, 데이터 일관성/정합성(consistency)은 상대적으로 더 세부적인 것을 본다고 보면 된다.
```



---
시니어 입장에서 Data Trust는 단순히 Data Integrity를 말하는 것이 아니라고 한다.



```
요소,설명,상혁의 정의와 비교
Integrity (무결성),데이터가 유실/중복/변조되지 않음,상혁이가 말한 핵심 (기본)
Freshness (최신성),데이터가 약속된 시간(SLA) 내에 도착함,처리 속도와 파이프라인 안정성
Validity (타당성),비즈니스 로직과 실제 도메인 규칙에 맞음,"dbt 테스트, 스키마 검증 등"
```

Data Trust는 단순히 '데이터가 망가지지 않는다'를 넘어 '**이 데이터를 근거로 의사결정을 내려도 비즈니스에 사고가 나지 않는다**'라고 확신을 주어야한다.

따라서, 위 3가지 조합이 보통 신뢰성을 나타낸다고 한다.






# 1. Data Trust 실습 진행하기



```
"DB에 기록된 총 거래액과 상혁님이 만들어준 데이터 웨어하우스(DW)의 총 합계가 0.01% 차이가 납니다. 이 상태로는 정산을 진행할 수 없습니다."
```

->이를 어떻게 해결할 것인가?

이 0.01%라는 숫자는 아주 작아 보이지만, 수천억 원 규모에서는 수천만원의 오차를 의미한다.

->이는 재무팀 입장에서는 절대 넘어가면 안 되는 수치이다.

->우리는 이 오차를 잡아내는 것을 넘어서, '**어느 단계에서 왜 발생했는지 입증(Audit)**'을 해야한다. 이를 위해 설계도를 그려볼 것이다.





# 1-1. Data Trust 확보를 위한 계층별 Audit 전략




## 1. Raw Layer(Bronze): "Source(원본)"와 똑같이 가져왔는가?


이 단계는 '**무결성(Integrity)**'의 기본을 체크하는 곳이다.



```
＊주요 Audit 항목:

1.Freshness Check: 데이터가 약속된 시간(SLA) 내에 들어왔는가? (예: max(created_at)이 현재 시간 기준 1시간 이내인가?)

2.Volume Check: 평소보다 데이터 유입량이 급격히 줄거나 늘지 않았는가?

3.Schema Check: Source DB에서 컬럼 타입이 바뀌지 않았는가?

※비즈니스 관점의 이유: 여기서 문제가 생기면 뒤의 모든 계산은 무의미하다. "우리가 잘못 계산한 게 아니라, 원천 데이터가 늦게 들어왔거나 유실되었습니다"라고 증거를 낼 수 있는 지점이다.
```


## 2. Silver Layer(Cleaned): "데이터 정상적인가?"

이 단계는 **데이터를 정제하고 join**하는 단계이다. '**일관성**' 정의가 가장 빛을 발해야 하는 곳이다.



```
＊주요 Audit 항목:

1.Primary Key Unique & Not Null: 분산 환경(Spark)에서 중복 적재가 발생하지 않았는가? (0.01% 오차의 주범 1위: 중복) -> 멱등성(Idempotency) 위반

2.Accepted Values: '결제상태' 컬럼에 'Wait', 'Success', 'Fail' 외에 이상한 값이 들어오지 않았는가?

3.Relationship Test: 결제 테이블에는 있는 ID가 회원 테이블에는 없는 '유령 회원' 데이터는 아닌가?

※비즈니스 관점의 이유: 데이터의 '형태'를 검증하는 단계이다. 여기서 중복만 잘 제거해도 재무팀이 말하는 오차의 80%는 해결된다.
```

## 3. Gold Layer(Business): "비즈니스 로직이 정답인가?"

이 단계는 최종 대시보드(관리자페이지)와 정산에 쓰이는 데이터이다. 여기서는 '**타당성(validity)**'가 핵심이다.



```
＊주요 Audit 항목:

1.Reconciliation (재무 대조): DW_SUM(amount) == SOURCE_SUM(amount). 원천 DB의 총 합계와 우리 마트의 합계를 비교해. (0.01% 오차를 직접적으로 찾아내는 곳)

2.Range/Constraint Test: 환불 금액이 결제 금액보다 큰가? (Negative check), 포인트 사용액이 결제 총액을 넘는가?

3.Floating Point Test: Double이나 Float 타입 연산 중 발생한 미세 오차인지, Decimal 타입을 써서 정확히 일치시켰는지 확인.

※비즈니스 관점의 이유: "데이터는 완벽하게 다 가져왔고 중복도 없지만, 로직상 이 오차는 환불 처리 방식의 차이 때문입니다"라고 설명할 수 있게 해준다.
```








# 1-2. 🧐시니어 관점의 압박 질문(Logic Battle) - ⭐Data Trust 질문들을 다 모아놓음.(오늘 프로젝트의 핵심)

**※내 질문/답변 및 시니어 관점에서의 Gemini 질문/답변을 원본 상태로 모아 놓음.**

**<Gemini의 시니어 관점 첫 질문>**

```
Q1. "0.01% 오차를 발견한 시점"의 문제 만약 dbt 테스트를 통해 Gold 레이어에서 0.01% 오차를 발견했어. 이미 데이터는 마트(Gold)까지 다 쌓인 상태야. 재무팀은 이미 그 데이터를 보고 "어? 틀렸네?"라고 불신을 가진 상태지.

상혁이라면, **"데이터를 다 쌓고 나서 테스트하는 방식"**과 "테스트를 통과하지 못하면 아예 데이터를 쌓지 않는 방식(Write-Audit-Publish)" 중 무엇을 택하겠어? 비용과 신뢰성 측면에서 비교해 봐.
```



```
Q2. "Late Arriving Data"의 함정 원천 DB에는 어제 밤 11시 59분 59초에 기록됐는데, 네트워크 지연으로 우리 DW에는 오늘 새벽 0시 5분 파이프라인에 포함되어 들어왔어. 그럼 어제 정산 데이터에는 이 금액이 빠지겠지? 0.01% 오차는 바로 이런 곳에서 생겨.

이 '시간 지연' 문제를 상혁이는 **'무결성'**의 관점에서 해결할래, 아니면 '재처리(Backfill)' 프로세스를 설계해서 해결할래?
```



---
**<내 답변>**



```
Q1의 경우 당연히 후자인 '테스트를 통과하지 못하면 아예 데이터를 쌓지 않는 방식(Write-Audit-Publish) 방식을 따라야지. 중간중간마다 안전장치가 있는 것과 없는 것은 천지차이니깐

Q2의 경우 일단 무결성 관점에서 처리하는 것은 어떻게 처리하는 거지? 재처리(Backfill)도 괜찮을 거 같은데.. 파티션으로 나눠져있고 dynamic 설정이라면, 그 날짜 부분만 바꾸면 되니깐. 근데 이것도 처리 비용이 많이 드려나 음..
```



---
**<Gemini Gems 3- 사고모드의 답변>**



```
1. Q1: WAP 패턴의 '진짜' 구현 (Logic Battle)
상혁아, "당연히 WAP를 써야지"라고 말했지만, dbt 환경에서 이걸 구현하는 건 생각보다 까다로워.

구현의 딜레마: 보통 dbt는 dbt run을 하면 테이블에 바로 데이터를 써버리잖아? 그 후에 dbt test를 하면 이미 데이터는 'Publish' 된 상태지.

시니어의 질문: 상혁이라면 이걸 어떻게 해결할래?

Staging Schema 활용: audit_db에 먼저 데이터를 쌓고 테스트를 돌린 뒤, 성공하면 production_db로 INSERT OVERWRITE 할래?

View & Table 분리: 데이터는 계속 쌓되, 테스트를 통과한 최신 Snapshot ID만 바라보는 'View'를 제공할래?

Blue-Green Deployment: 아예 테이블 두 개를 번갈아 가며 쓸래?

힌트: '비용'과 '원자성(Atomicity)'을 생각해 봐. 10TB의 데이터를 매번 INSERT OVERWRITE 하는 게 효율적일까? 아니면 Iceberg 같은 최신 포맷의 Snapshot 기능을 활용해 볼까?
```




```
2. Q2: '무결성 관점'의 처리 vs '재처리(Backfill)'
상혁이가 물어본 "무결성 관점에서 처리하는 것"은 **'사후 약방문'이 아니라 '설계 단계의 예방'**을 의미해. 0.01%의 지연 데이터(Late Arriving Data) 문제를 해결하는 두 가지 철학을 비교해 줄게.

A. 무결성 관점 (Pro-active Design)
핵심: Event Time과 Processing Time을 철저히 분리하는 거야.

방법: 1. Watermark(워터마크) 설정: "데이터는 최대 1시간 늦게 올 수 있다"라고 가정하고, 배치 처리를 할 때 항상 T-1시간부터 T까지의 데이터를 처리하는 게 아니라, 지연을 고려해 윈도우를 잡는 거지. 2. Idempotency(멱등성) 보장: 상혁이가 말한 '파티션 교체' 방식이야. Unique Key를 기준으로 중복을 제거하는 MERGE 문을 사용해서, 늦게 온 데이터가 들어와도 전체 합계가 깨지지 않게 설계하는 거야.

B. 재처리 관점 (Re-active Backfill)
핵심: 일단 쌓고, 오차가 발견되면 다시 돌린다.

상혁의 우려(비용): 맞어, 1년 치 데이터를 다시 돌리면 비용 폭탄이지.

시니어의 질문: * 상혁아, 만약 어제 데이터에 오차가 생겼는데, 그 오차가 오늘 정산에 영향을 준다면 언제까지 기다렸다가 Backfill을 돌릴 거야?

매번 사람이 수동으로 "오차 있네? 다시 돌려야지"라고 하는 시스템을 '신뢰'할 수 있을까?
```



---
**<답변에 대한 내 추가 답변>**



```
Q1의 경우, 너의 힌트에 힘 입어서, 1번 staging schema를 활용하면, 원자성 개념으로 하나라도 실패하면 전부 다 실패일 거 아니야? 그래서 뭔가 좀 시간이 오래걸릴 거 같고, 2번 view & table 분리는, 괜찮아보이는데, view라는 게 일반적으로 데이터를 저장하지 않으니깐.. 테스트 통과한 최신 snapshot id는 어떻게 가져오는 거지? 그 부분만 table에서 가져오는 건가? 3번 Blue-Green Deploymnet는 설명이 아예 테이블 2개를 번갈아가면서 쓴다는 것 밖에 없는데, 이 의미는 2번의 뷰 방식을 좀 더 빠르게 처리하는 건가?

Q2의 경우, A.무결성 관점에서, 1번의 Watermark 설정하는 거 좋아보이는데, 만약 어떤 데이터가 1시간 보다 넘어선다면? 2번은 마치 upsert 같이 처리하는 거 같은데, 1번과 2번을 같이 사용해서 잘 활용하면 좋을 거 같은데 뭔가.. B.재처리 관점은 개인적으로 최악인 거 같아.
```



---
**<내 추가 답변에 대한 Gemini의 추가 답변>**



```
1. Q1: WAP 패턴의 구현 - "어디에 저장하고 어떻게 보여줄 것인가?"
A. 1번(Staging)의 한계: 상혁이 말대로 10TB 데이터를 Raw -> Staging -> Prod로 두 번 쓰는 건 시간과 비용(Storage, Slot) 낭비야. 하지만 '원자성(Atomicity)'은 확실하지. "전부 성공하거나, 아예 안 보이거나." B. 2번(View & Table)의 비밀: "Snapshot ID를 어떻게 가져오지?"라는 질문은 아주 날카로워. 보통 'Metadata DB' 혹은 **'Control Table'**을 따로 둬.

control_table: (table_name, last_success_snapshot_id, updated_at)

View 쿼리: SELECT * FROM real_table WHERE snapshot_id = (SELECT last_success_snapshot_id FROM control_table WHERE ...)

장점: 데이터는 딱 한 번만 쓰면 돼. 테스트가 통과하는 순간 control_table의 ID만 '툭' 바꿔주면 끝이지.

C. 3번(Blue-Green): 이건 2번의 '물리적 확장판'이야. 아예 table_A, table_B 두 개를 만들어 놓고, 이번 배치는 A에 써서 검증해. 통과하면 서비스 View가 바라보는 방향을 B에서 A로 ALTER TABLE RENAME 시켜버리는 거야. 메타데이터만 바꾸는 거라 0.1초 만에 끝나지.

🧐 시니어의 질문: 상혁아, 만약 상혁이가 BigQuery나 Snowflake 같은 최신 DW를 쓴다면, 이런 수동적인 구조 대신 **'Iceberg의 Snapshot'**이나 'Clone' 기능을 써서 비용을 0원에 가깝게 줄이면서 WAP를 구현할 수 있을까? 어떻게 하면 될지 상상해 봐.
```




```
2. Q2: Late Arriving Data - "정답을 언제까지 기다릴 것인가?"
상혁이가 제안한 'Watermark(1번) + Upsert(2번)' 조합은 실무에서 가장 많이 쓰이는 정석이야. 하지만 상혁이가 우려한 "1시간 넘게 늦게 오면?" 이라는 상황이 엔지니어를 괴롭히는 진짜 변수지.

A. 무결성의 '데드라인' 설정: 1시간 워터마크를 설정했는데 2시간 뒤에 데이터가 왔어. 시스템은 이미 1시간 전 데이터를 '확정'하고 정산을 끝냈지.

시니어의 시각: 이건 에러가 아니라 **'비즈니스 정책'**의 영역이야. "우리 시스템은 T+1시간까지만 보정해 준다"라고 선언하는 거지. 그 이후 데이터는 '익일 정산'으로 넘기거나 별도 로그로 쌓아.

B. Upsert(Idempotency)의 비용: Upsert(MERGE)는 아주 강력하지만, 매번 전체 테이블을 뒤져서 ID를 대조해야 하니 데이터가 커질수록 컴퓨팅 비용이 기하급수적으로 늘어.
```



```
🚀 오늘의 하이라이트: "0.01%의 유령을 어떻게 다룰 것인가?"
자, 이제 상혁이의 논리를 마무리 짓는 마지막 퍼즐이야.

상혁이가 Watermark + Upsert + dbt Test를 다 도입했어. 그런데도 재무팀은 여전히 **"0.01% 오차"**를 들고 와. 확인해 보니, 이건 시스템 오류가 아니라 **"어제 밤에 결제했는데, 은행 망 점검 때문에 오늘 아침에 승인 완료된 건"**들이었어. 즉, 원천 DB와 우리 DW의 '기준 시간' 차이 때문에 발생하는 **'피할 수 없는 오차'**였던 거야.

상혁이라면 이 상황에서 어떤 선택을 하겠어?

**"이건 정상입니다"**라고 매일 재무팀에게 메일을 보낸다.

"Tolerance(허용 오차) 설정": dbt 테스트에 error_if: > 0.05% 같은 임계치를 둬서, 미세한 차이는 자동으로 통과시키고 시스템이 멈추지 않게 한다.

"Reconciliation Table": 매일 발생한 오차 금액만 따로 모으는 테이블을 만들어서, 다음 날 자동으로 합쳐지게 설계한다.

💡 상혁아, 어떤 방식이 '가장 적은 운영 공수'로 '가장 높은 신뢰'를 줄 수 있을까? 특히 상혁이가 말했던 **"원치 않는 상황(오차)도 무덤덤하게 받아들이는 자세"**를 시스템적으로 어떻게 구현할지 고민해 봐.
```


___
**<Gemini 추가 답변에 대한 내 추가 답변 - 2>**



```
장점: 데이터는 딱 한 번만 쓰면 돼. 테스트가 통과하는 순간 control_table의 ID만 '툭' 바꿔주면 끝이지.

이 부분에서 ID를 어떻게 바꿔주는 지를 설명해줬으면 좋겠어.

C. 3번(Blue-Green):
이건 2번의 '물리적 확장판'이야. 아예 table_A, table_B 두 개를 만들어 놓고, 이번 배치는 A에 써서 검증해. 통과하면 서비스 View가 바라보는 방향을 B에서 A로 ALTER TABLE RENAME 시켜버리는 거야. 메타데이터만 바꾸는 거라 0.1초 만에 끝나지.

이 부분도, 그냥 단순 테이블만 바꾸는 건지? 이러면 만약에 table b에 오류가 생기면 어떻게 할 건지에 대한 설명도

🧐 시니어의 질문: 상혁아, 만약 상혁이가 BigQuery나 Snowflake 같은 최신 DW를 쓴다면, 이런 수동적인 구조 대신 **'Iceberg의 Snapshot'**이나 'Clone' 기능을 써서 비용을 0원에 가깝게 줄이면서 WAP를 구현할 수 있을까? 어떻게 하면 될지 상상해 봐.

내가 이런 기능들을 안 써봐서 어떤 건지 잘 모르겠어.

🚀 오늘의 하이라이트: "0.01%의 유령을 어떻게 다룰 것인가?"

이 부분에 대한 답변은,
**"이건 정상입니다"**라고 매일 재무팀에게 메일을 보낸다. -> 이건 pass. 별로야. 자동화 로직으로 처리할 수가 없잖아. DX 관점에서 out이야.
"Tolerance(허용 오차) 설정": dbt 테스트에 error_if: > 0.05% 같은 임계치를 둬서, 미세한 차이는 자동으로 통과시키고 시스템이 멈추지 않게 한다. -> 과연.. 이 오차의 임계치로 또 다른 에러가 안 생길까?
"Reconciliation Table": 매일 발생한 오차 금액만 따로 모으는 테이블을 만들어서, 다음 날 자동으로 합쳐지게 설계한다. -> 차라리 이게 제일 난 거 같은데.. 업데이트는 좀 늦어져도 ㅇㅇ..
```



___
**<내 추가 답변에 대한 Gemini의 추가 답변 - 2>**



```
1. Control Table의 ID를 어떻게 '툭' 바꾸는가?
이건 비유하자면 **'포인터'**를 옮기는 것과 같아.

구조: control_table은 현재 서비스가 바라봐야 할 최신 Snapshot ID나 날짜를 들고 있는 아주 작은 테이블이야.

작동 원리:

data_table에 데이터를 밀어 넣어. (이때 데이터에는 snapshot_id = '20240522_v1' 같은 꼬리표가 붙어있어)

이 snapshot_id를 대상으로 dbt test를 돌려.

테스트가 통과하면, SQL 한 줄을 날려: UPDATE control_table SET current_snapshot_id = '20240522_v1' WHERE table_name = 'daily_settlement';

사용자가 보는 View는 항상 JOIN control_table을 하고 있어서, 이 UPDATE가 일어나는 순간 즉시 새로운 데이터를 보게 돼.
```



```
2. Blue-Green Deployment와 장애 대응
상혁이가 걱정한 "table_b에 오류가 생기면?"이라는 상황이 바로 이 방식의 최대 장점이야.

시나리오: 현재 서비스는 table_A(Blue)를 보고 있어. 상혁이가 새로 만든 데이터는 table_B(Green)에 담겼지.

검증 단계: table_B에 대해 dbt test를 돌렸는데 실패했어!

대응: 그러면 ALTER TABLE RENAME 명령어를 실행하지 않아. * 결과: 서비스는 여전히 안전한 table_A를 바라보고 있지. 즉, 오류가 있는 데이터(B)가 사용자에게 노출되는 것을 원천 차단하는 거야. "데이터가 좀 늦게 업데이트되는 것(Freshness)"이 "틀린 데이터가 나가는 것(Integrity)"보다 낫다는 판단이지.
```



```
3. 최신 DW의 마법: Iceberg Snapshot & Zero-copy Clone
이 기능들을 안 써봤다면 생소하겠지만, 원리는 간단해. **"데이터를 복사하지 않고 이름표만 붙인다"**는 거야.

Iceberg Snapshot (Git과 비슷함): 테이블에 데이터를 추가할 때마다 Git의 커밋처럼 '스냅샷'이 생성돼. "상혁아, 1번 스냅샷(과거)으로 돌아가!"라고 명령하면 1초 만에 테이블이 과거 상태로 돌아가. WAP를 할 때 Audit이 실패하면 그냥 Rollback to Snapshot 명령 한 줄이면 끝나는 거지.

Snowflake Clone (Zero-copy Clone): 10TB짜리 테이블을 복사(Clone)해도 저장 공간을 차지하지 않아. 원본 데이터 블록을 그대로 가리키는 **'복제본 이름표'**만 하나 더 만드는 거거든.

PROD 테이블을 STAGE로 순식간에 Clone 한다.

STAGE에서 데이터를 수정하고 테스트한다.

테스트 통과하면 STAGE와 PROD를 SWAP 한다. 이 모든 게 메타데이터만 건드리는 거라 비용이 0원이고 속도는 빛보다 빠르지.
```




```
4. 0.01% 유령 데이터 대응: "Reconciliation Table"
오, 상혁이의 선택(3번)이 아주 훌륭해! 나도 실무에서 이 방식을 가장 선호해. 왜냐하면 이게 가장 'Self-healing(자가 치유)' 모델에 가깝거든.

상혁의 논리 분석: * 1번(메일 발송)은 엔지니어를 피곤하게 해.

2번(오차 허용)은 진짜 터져야 할 에러를 숨길 위험이 있어.

3번(재대조 테이블): "오늘 발생한 차액은 '지연 데이터'로 분류해서 일단 따로 보관하고, 내일 데이터가 들어오면 합친다." -> 이건 **최종적인 일관성(Eventual Consistency)**을 보장하는 아주 성숙한 설계야.
```




```
🧐 시니어의 마지막 압박 질문 (Final Mission)
상혁아, 이제 'Data Trust'의 모든 이론과 실무 전략을 꿰뚫었어. 마지막으로 당근(Daangn) 면접관이 던질 법한 질문 하나만 더 해볼게.

"상혁님, Reconciliation Table 방식을 쓰면 데이터는 결국 맞게 되겠지만, 재무팀 대시보드에는 '어제 매출'이 매일 아침 조금씩 변하게(Update) 될 텐데, 이걸 사용자(재무팀)에게 어떻게 납득시킬 건가요?

'데이터가 매일 변하는 시스템'을 신뢰하게 만들 수 있는 커뮤니케이션 혹은 시스템적 장치가 있을까요?"
```


___
**<마지막 내 답변>**




```
"상혁님, Reconciliation Table 방식을 쓰면 데이터는 결국 맞게 되겠지만, 재무팀 대시보드에는 '어제 매출'이 매일 아침 조금씩 변하게(Update) 될 텐데, 이걸 사용자(재무팀)에게 어떻게 납득시킬 건가요?
'데이터가 매일 변하는 시스템'을 신뢰하게 만들 수 있는 커뮤니케이션 혹은 시스템적 장치가 있을까요?"

몰라 일단은 난 저거 선택하면서, 재무팀에게 다음 날 재무 업데이트가 변경될 수 있음을 공지하는 것만 생각했는데
```


**<마지막 Gemini의 답변>**



```
🛠️ 데이터가 변해도 신뢰를 얻는 3가지 시스템 설계
상혁이가 제안한 'Reconciliation Table'을 쓰면서, 재무팀의 불안감을 해소할 수 있는 **'시스템적 장치'**들을 제안해 줄게.

1. 데이터 상태(Status) 값 도입: "Draft vs Final"
데이터에 단순히 숫자만 넣지 말고, is_finalized 혹은 data_status 같은 컬럼을 추가하는 거야.

T일 아침: "어제 매출: 10억 (상태: 미확정/Draft)" -> "아직 지연 데이터가 다 안 들어왔으니 참고용으로만 보세요"라는 신호를 시스템이 주는 거지.

T+1일 아침: "어제 매출: 10억 100만 (상태: 확정/Final)" -> "이제 모든 대조가 끝났으니 이 숫자로 정산하세요"라고 확신을 주는 거야.

2. 오차의 가시화 (Transparency Dashboard)
"숫자가 바뀌었습니다"라고 말만 하지 말고, 얼마나 바뀌었는지를 보여주는 '오차 리포트'를 대시보드 한 구석에 띄워줘.

"오늘 추가된 어제 지연 데이터: 0.01% (1,000만 원)"

이렇게 오차의 원인과 규모를 투명하게 공개하면, 재무팀은 "아, 상혁님 시스템이 지연 데이터를 놓치지 않고 끝까지 다 잡아내고 있구나"라며 오히려 더 신뢰하게 돼.

3. 마감 시점(SLA) 정의
"무한정 데이터가 바뀌는 게 아니라, 배치 발생 후 48시간까지만 보정합니다"라는 규칙(SLA)을 기술적으로 세팅해. 그 이후에 들어오는 건 '기타 보정분'으로 따로 처리하는 거지.
```









# 2. 위 1-2의 장황한 설명 핵심 가이드라인

## 1. <Gemini의 시니어 관점 첫 질문> - 2가지 질문



```
1. 데이터 다 쌓고 테스트 or 테스트 통과 x -> 쌓지 x
2. 시간 지연에 관한 문제 -> 무결성 해결 or 재처리(Backfill)
```

## 2. <내 답변> - 위 2가지 질문에 대한 나의 답변


```
1. WAP 방식 채용(테스트 통과 x -> 데이터 쌓지 x)
2. 무결성 처리 방식은 어떻게 되냐? 재처리(Backfill)도 partition & dynamic 설정이면 괜찮아 보인다.
```



## 3. <Gemini Gems 3- 사고모드의 답변> - 위 나의 답변에 대한 Gemini의 보충 설명 답변 및 추가 질문



```
1. WAP 패턴의 '진짜(현실적으로)' 구현은 어렵다. -> dbt 환경에서 이걸 구현하기는 생각보다 까다롭다.

2. 시니어 질문: 이걸 어떻게 해결하겠나?
-> 1. Staging Schema 활용
-> 2. View & Table 분리
-> 3. Blue-Green Deployment

3. 무결성 관점 vs 재처리 관점에 대한 설명
```

## 4. <답변에 대한 내 추가 답변> - 위 Gemini 답변에 대한 나의 추가 답변



```
1-1. staging schema 활용하면, 원자성 개념으로 하나라도 실패하면 전부 다 실패 -> 시간이 오래걸릴 거 같다.
1-2. view & table 분리 괜찮아보이지만, view는 일반적으로 데이터 저장 x -> 테스트 통과한 최신 snapshot id는 어떻게 가져오느냐?
1-3. Blue-Green Deploymnet는 2번의 뷰 방식을 좀 더 빠르게 처리하는 거냐?

2-1. 무결성 관점에서, 1번의 Watermark 방식 좋아보이는데, 만약 해당 시간을 오버한다면?
2-2. 마치 upsert 같이 처리하는 거 같은데, 1번과 2번을 같이 활용하면 좋아보일 듯하다.
2-3. Backfill은 개인적으로 최악이라고 생각한다.
```


## 5. <내 추가 답변에 대한 Gemini의 추가 답변> - 위 내 답변에 대한 Gemini의 추가 답변 및 질문



```
1. 내 답변에 대한 평가 및 답변(설명 요구) 및 시니어 관점 추가 질문(BigQuery나 Snowflake 같은 최신 DW를 쓴다면, 이런 수동적인 구조 대신 **'Iceberg의 Snapshot'**이나 'Clone' 기능을 써서 비용을 0원에 가깝게 줄이면서 WAP를 구현할 수 있을까?)

2. 내 답변에 대한 평가

3. 하이라이트 추가 질문
```

## 6. <Gemini 추가 답변에 대한 내 추가 답변 - 2> - 이어서..



```
1. 내가 모르는 부분 질문

2. 위 1번 시니어 관점 추가 질문에 대한 내가 이런 기능을 안 써서 답을 못 함.

3. 오늘의 하이라이트에 대한 내 답변("Reconciliation Table"이 제일 나아보인다.)
```

## 7. <내 추가 답변에 대한 Gemini의 추가 답변 - 2> - 이어서..



```
1. 내가 모르는 부분 질문에 대한 답변

2. 오늘의 하이라이트에 대한 내 답변 평가

3. 시니어의 마지막 압박 질문("상혁님, Reconciliation Table 방식을 쓰면 데이터는 결국 맞게 되겠지만, 재무팀 대시보드에는 '어제 매출'이 매일 아침 조금씩 변하게(Update) 될 텐데, 이걸 사용자(재무팀)에게 어떻게 납득시킬 건가요?

'데이터가 매일 변하는 시스템'을 신뢰하게 만들 수 있는 커뮤니케이션 혹은 시스템적 장치가 있을까요?")
```

## 8. <마지막 내 답변>



```
1. 일단은 단순히 다음 날 재무팀에 공지함을 생각함.

```

## 9. <마지막 Gemini의 답변>



```
1. 내 답변에 대한 Gemini의 평가

2. 대안 3가지 제안
-> 1. 데이터 상태(Status) 값 도입
-> 2. 오차의 가시화 (Transparency Dashboard)
-> 3. 마감 시점(SLA) 정의
```












# 3. 결론 및 마무리

예상치 못하게 따로 실습 작업은 이루어지지 않았다.

실습을 통해서 정확히 상황을 파악하는 것이 아닌 단순 텍스트로 상황을 가정하고, 모르는 부분을 질문하고 답변 받고...

이런 식으로 진행해서 그런지 상당히 많은 양의 정보들이 들어와서 한번에 이해하기에는 약간의 어려움이 따랐고,

아직은 익숙하지 않은 느낌이다(당연하지만..)

하지만, 이번에도 Data Trust 실습을 통해,

단순 로직(논리)적인 차원에서 갇혀서 생각하는 게 아닌,

**다른 실습때보다 본격적으로 Business 관점으로 바라볼 수 있는 계기가 되어서 좋은 경험이었던 거 같다.**

